
# Agentic AI Assistant for ShopUNow  



In [1]:

!pip install -q langgraph langchain langchain-openai chromadb textblob


In [2]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph, END
from typing import Literal
from pydantic import BaseModel, Field


In [8]:
from dotenv import load_dotenv
import os
# Use override=True to enforce reloading from .env, fixing any stale 'OPENAI_API_KEY' set previously
load_dotenv(override=True)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings()

key = os.getenv("OPENAI_API_KEY")


In [9]:

class DepartmentOutput(BaseModel):
    department: Literal[
        "HR",
        "IT_SUPPORT",
        "BILLING",
        "SHIPPING",
        "UNKNOWN"
    ] = Field(description="Classified department")


class SentimentOutput(BaseModel):
    sentiment: Literal[
        "POSITIVE",
        "NEUTRAL",
        "NEGATIVE"
    ] = Field(description="Sentiment of the user query")


## 1. Department Definitions

In [10]:

DEPARTMENTS = {"HR","IT_SUPPORT","BILLING","SHIPPING"}

## 2. Unified Dataset (All Departments – 60 Q&A)

In [11]:

all_faqs = [
    # ---------------- HR (15) ----------------
    {"doc": "Q: How do I apply for paid leave? A: Apply through the HR portal.", "category": "HR"},
    {"doc": "Q: When is salary credited? A: On the last working day of the month.", "category": "HR"},
    {"doc": "Q: How can I download my payslips? A: From the Payroll section in HR portal.", "category": "HR"},
    {"doc": "Q: What is the probation period? A: Six months from date of joining.", "category": "HR"},
    {"doc": "Q: What is the notice period? A: 30 days unless stated otherwise.", "category": "HR"},
    {"doc": "Q: How do I update bank details? A: Submit a request via HR portal.", "category": "HR"},
    {"doc": "Q: How does performance appraisal work? A: Conducted annually based on goals.", "category": "HR"},
    {"doc": "Q: How do I apply for work from home? A: Through HR system with manager approval.", "category": "HR"},
    {"doc": "Q: How do I apply for internal transfer? A: Apply via internal job portal.", "category": "HR"},
    {"doc": "Q: What benefits are offered? A: Health insurance, leaves, bonuses.", "category": "HR"},
    {"doc": "Q: How do I raise a grievance? A: Use HR grievance system confidentially.", "category": "HR"},
    {"doc": "Q: What is attendance regularization? A: Correct missed attendance entries.", "category": "HR"},
    {"doc": "Q: Who handles tax queries? A: HR payroll team.", "category": "HR"},
    {"doc": "Q: How do I update personal details? A: Update directly in HR portal.", "category": "HR"},
    {"doc": "Q: What is the resignation process? A: Submit resignation via HR system.", "category": "HR"},

    # ---------------- IT SUPPORT (15) ----------------
    {"doc": "Q: How do I reset my password? A: Use IT self-service portal.", "category": "IT_SUPPORT"},
    {"doc": "Q: VPN not working? A: Restart and raise IT ticket.", "category": "IT_SUPPORT"},
    {"doc": "Q: Laptop running slow? A: Close apps and contact IT.", "category": "IT_SUPPORT"},
    {"doc": "Q: How do I install software? A: Use company software center.", "category": "IT_SUPPORT"},
    {"doc": "Q: Email not accessible? A: Raise IT support ticket.", "category": "IT_SUPPORT"},
    {"doc": "Q: How do I access shared drives? A: Request permission from IT.", "category": "IT_SUPPORT"},
    {"doc": "Q: What to do if laptop is damaged? A: Inform IT immediately.", "category": "IT_SUPPORT"},
    {"doc": "Q: How do I connect to office WiFi? A: Use employee credentials.", "category": "IT_SUPPORT"},
    {"doc": "Q: How to report phishing email? A: Use Report Phishing option.", "category": "IT_SUPPORT"},
    {"doc": "Q: What is BitLocker recovery? A: Contact IT support.", "category": "IT_SUPPORT"},
    {"doc": "Q: How often change password? A: Every 90 days.", "category": "IT_SUPPORT"},
    {"doc": "Q: How to set email on phone? A: Follow IT mobile setup guide.", "category": "IT_SUPPORT"},
    {"doc": "Q: How do I request new laptop? A: Submit IT asset request.", "category": "IT_SUPPORT"},
    {"doc": "Q: Account locked? A: Contact IT support.", "category": "IT_SUPPORT"},
    {"doc": "Q: How to backup work files? A: Use approved cloud storage.", "category": "IT_SUPPORT"},

    # ---------------- BILLING (15) ----------------
    {"doc": "Q: What payment methods are accepted? A: Cards, UPI, net banking.", "category": "BILLING"},
    {"doc": "Q: Payment failed but deducted? A: Auto-refund in 3–5 days.", "category": "BILLING"},
    {"doc": "Q: Refund timeline? A: 5–7 business days.", "category": "BILLING"},
    {"doc": "Q: Where can I download invoice? A: From My Orders.", "category": "BILLING"},
    {"doc": "Q: Charged twice? A: One amount will be refunded.", "category": "BILLING"},
    {"doc": "Q: How to apply promo code? A: Apply during checkout.", "category": "BILLING"},
    {"doc": "Q: Is COD available? A: Available on select products.", "category": "BILLING"},
    {"doc": "Q: Can payment method be changed? A: No after order placement.", "category": "BILLING"},
    {"doc": "Q: Refund credited partially? A: Discounts or fees applied.", "category": "BILLING"},
    {"doc": "Q: How to check payment status? A: In Order Details.", "category": "BILLING"},
    {"doc": "Q: Are there extra charges? A: Taxes shown at checkout.", "category": "BILLING"},
    {"doc": "Q: How to cancel order? A: Cancel from My Orders.", "category": "BILLING"},
    {"doc": "Q: Why promo code failed? A: May be expired.", "category": "BILLING"},
    {"doc": "Q: How to contact billing support? A: Raise support ticket.", "category": "BILLING"},
    {"doc": "Q: When will refund reflect? A: Depends on payment mode.", "category": "BILLING"},

    # ---------------- SHIPPING (15) ----------------
    {"doc": "Q: How can I track my order? A: Track via My Orders.", "category": "SHIPPING"},
    {"doc": "Q: Delivery delayed? A: Check updated timeline.", "category": "SHIPPING"},
    {"doc": "Q: What are delivery timelines? A: 3–7 business days.", "category": "SHIPPING"},
    {"doc": "Q: Can I change delivery address? A: Before shipment only.", "category": "SHIPPING"},
    {"doc": "Q: Received damaged product? A: Initiate return.", "category": "SHIPPING"},
    {"doc": "Q: How does return pickup work? A: Pickup scheduled automatically.", "category": "SHIPPING"},
    {"doc": "Q: Missed delivery attempt? A: Courier will retry.", "category": "SHIPPING"},
    {"doc": "Q: Same-day delivery available? A: In select cities.", "category": "SHIPPING"},
    {"doc": "Q: Order shows delivered but not received? A: Contact support.", "category": "SHIPPING"},
    {"doc": "Q: Shipping charges? A: Based on order value.", "category": "SHIPPING"},
    {"doc": "Q: Can I cancel after shipping? A: Refuse delivery.", "category": "SHIPPING"},
    {"doc": "Q: How to reschedule delivery? A: From tracking page.", "category": "SHIPPING"},
    {"doc": "Q: International delivery supported? A: On select products.", "category": "SHIPPING"},
    {"doc": "Q: Delivery slot selection? A: Available at checkout.", "category": "SHIPPING"},
    {"doc": "Q: How to contact delivery support? A: Use Help section.", "category": "SHIPPING"},
]


## 3. Create Documents & Embeddings

In [12]:

documents = [
    Document(page_content=item["doc"], metadata={"department": item["category"]})
    for item in all_faqs
]

vectordb = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory="chroma_db"
)
vectordb.persist()

print(f"Created embeddings for {len(documents)} documents")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## 4. LangGraph Agent

In [9]:

def sentiment_node(state):
    sentiment_llm = llm.with_structured_output(SentimentOutput)

    result = sentiment_llm.invoke(
        f"Classify sentiment (positive, neutral, negative): {state['query']}"
    )

    state["sentiment"] = result.sentiment
    return state


def classify_node(state):
    dept_llm = llm.with_structured_output(DepartmentOutput)

    result = dept_llm.invoke(
        f"Classify department (HR, IT_SUPPORT, BILLING, SHIPPING, UNKNOWN): {state['query']}"
    )

    state["department"] = result.department
    return state


def rag_node(state):
    docs = vectordb.similarity_search(state["query"], k=3, filter={"department": state["department"]})
    context = "\n".join(d.page_content for d in docs)
    state["response"] = llm.invoke(f"Context:\n{context}\nQuestion:{state['query']}").content
    return state

def human_node(state):
    state["response"] = "A human agent will contact you shortly."
    return state


In [10]:

graph = StateGraph(dict)
graph.add_node("sentiment", sentiment_node)
graph.add_node("classify", classify_node)
graph.add_node("rag", rag_node)
graph.add_node("human", human_node)
graph.set_entry_point("sentiment")
graph.add_edge("sentiment","classify")

def route(state):
    return "human" if state["sentiment"]=="negative" or state["department"] not in DEPARTMENTS else "rag"

graph.add_conditional_edges("classify", route, {"rag":"rag","human":"human"})
graph.add_edge("rag",END)
graph.add_edge("human",END)
agent = graph.compile()


## 5. Test Queries

In [11]:

queries = [
    "My salary was not credited and I am angry",
    "How do I reset my VPN password?",
    "Where is my order?",
    "How to contact delivery support?"
]

for q in queries:
    print("\nQuery:", q)
    print(agent.invoke({"query": q}))



Query: My salary was not credited and I am angry
{'query': 'My salary was not credited and I am angry', 'sentiment': 'NEGATIVE', 'department': 'BILLING', 'response': 'A: I understand your frustration. Please check with your payroll department to ensure there were no issues with the payment process. If there’s an error, they should be able to assist you in resolving it.'}

Query: How do I reset my VPN password?
{'query': 'How do I reset my VPN password?', 'sentiment': 'NEUTRAL', 'department': 'IT_SUPPORT', 'response': 'A: Use the IT self-service portal to reset your VPN password.'}

Query: Where is my order?
{'query': 'Where is my order?', 'sentiment': 'NEUTRAL', 'department': 'SHIPPING', 'response': 'A: You can check the status of your order by visiting the My Orders section.'}

Query: How to contact delivery support?
{'query': 'How to contact delivery support?', 'sentiment': 'NEUTRAL', 'department': 'SHIPPING', 'response': 'A: Use the Help section.'}
